In [41]:
import os
import json
import IPython
from scipy.io import wavfile
import multiprocessing
import pandas as pd
from tqdm import tqdm
import operator

In [3]:
def get_transcription(path2transcription):
    transcription = []
    offset = -1
    end = 0
    with open(path2transcription, 'rt') as f:
        records = f.readlines()
        for sentence in records:
            fields = sentence.split()
            label = fields[6:]
            if not 'ignore_time_segment_in_scoring' in label:
                if offset < 0:
                    offset = float(fields[3])
                transcription.extend(label)
                end = float(fields[4])
    transcription = " ".join(transcription)
    duration = end - offset
    return offset, duration, transcription

def get_offset(path2transcription):
    
    with open(path2transcription, 'rt') as f:
        records = f.readlines()
        for sentence in records:
            fields = sentence.split()
            label = fields[6:]
            if not 'ignore_time_segment_in_scoring' in label:
                return float(fields[3])
    return 

In [27]:
def get_transcription_train(path2transcription):
    sentences = []
    with open(path2transcription, 'rt') as f:
        records = f.readlines()
        for record in records:
            fields = record.split()
            label = fields[6:-1]
            start, end = float(fields[3]), float(fields[4])
            label = [word[:-3] if word.endswith(')') else word for word in label]
            label = " ".join(label)
            
            sentences.append((start, end, label))
    sentences = sorted(sentences, key=lambda x: x[0])
    return sentences

def get_offset_train(path2transcription):
    offsets = []
    with open(path2transcription, 'rt') as f:
        records = f.readlines()
        for record in records:
            fields = record.split()
            offsets.append(float(fields[3]))
            
    return min(offsets)

In [20]:
sentences = get_transcription_train('/aimlx/Datasets/TEDLIUM_release1/train/stm/JessaGamble_2010GU.stm')


In [22]:
l = []
l.extend([5]*3)


In [28]:
off = get_offset_train('/aimlx/Datasets/TEDLIUM_release1/train/stm/JehaneNoujaim_2006.stm')



In [29]:
off


16.69

In [9]:
fs, signal = wavfile.read('/aimlx/Datasets/TEDLIUM_release1/train/wav/SheenaIyengar_2010G.wav')
start = int(fs * 513.11)
end = int(fs * 513.46)

In [10]:
IPython.display.Audio(signal[start:end], rate=fs)

In [5]:
pwd

'/aimlx/kws/kws_db_creation'

In [75]:
path = '/aimlx/Datasets/TEDLIUM_release1/dev/json/BlaiseAguerayArcas_2007.json'  
def analyze_jsons():
    with open('/aimlx/Datasets/TEDLIUM_release1/dev/json/BlaiseAguerayArcas_2007.json') as json_file:
        data = json.load(json_file)
        nb_words = len(data['words'])
        nb_aligned = 0
        unaligned_words = []
        for word in data['words']:
            if word['case'] == 'success':
                if word['word'] == 'computationally':
                    print(word['start'], word['end'])
                nb_aligned += 1
            if word['case'] != 'success':
                unaligned_words.append(word['word'])
        print('Percentage of aligned words', 100*(nb_aligned/nb_words))
        print('Unaligned words : ', unaligned_words)

In [17]:
fs, signal = wavfile.read('/aimlx/Datasets/TEDLIUM_release1/train/wav/AaronHuey_2010X.wav')
#start = int(fs * (225.09 + offset))
#end = int(fs * (225.85 + offset))
#microsoft = signal[start:end]

In [76]:
analyze_jsons()

225.09 225.85
Percentage of aligned words 94.885598923284
Unaligned words :  ['what', 'i', "'m", 'going', "'s", 'an', 'kind', "'t", 'it', "'s", 'in', 'the', 'doesn', 'you', 'that', 'it', "'s", 'that', 'it', "'s", "'t", "'s", 'you', 'want', "'t", 'now', 'a', 'graduate', 'student', 'at', 'kind', 'the', 'we', 'they', "'ve", 'and', 'know', 'it', "'s", 'very', 'it', "'s", 'to', 'this', 'kind', 'is', 'this', 'is', 'in', 'many', 'of', 'them', 'are', 'occluded', 'by', 'faces', 'and', 'and', 'so', 'of', 'of', 'of', 'do', 'i', 'do', 'i', 'understand', 'what', 'that', 'and', 'it', 'and', 'and', 'that', 'so', 'much']


In [50]:
offset

12.2

In [52]:
start

115372.2

In [19]:
def build_keyword_database(keywords, path2jsons, path2kw_db):
    return


def extract_kw(keyword, path2jsons, path2kw_db):
    
    if not os.path.exists(os.path.join(path2kw_db, keyword)):
        os.mkdir(os.path.join(path2kw_db, keyword))
        
    for path in path2jsons:
        with open(path) as json_file:
            data = json.load(json_file)
            count = 0
            for word in data['words']:
                if word['word'] == keyword and word['case'] == 'success':
                    start, end = word['start'], word['end']
                    offset = get_offset(path2transcription=path.replace('json', 'stm'))
                    fs, signal = wavfile.read(path.replace('json', 'wav'))
                    start, end = int(fs * (start + offset)), int(fs * (end + offset))
                    filename = path.split('/')[-1].split('.')[0] + '_' + str(count) + '.wav'
                    wavfile.write(os.path.join(path2kw_db, keyword, filename), data=signal[start:end], rate=fs)
                    count += 1 
                    
def get_jsons(path='/aimlx/Datasets/TEDLIUM_release1/dev/json'):
    
    path2jsons = []
    for _, _, files in os.walk(path):
        for file in files:
            if file.endswith('.json') and 'checkpoint' not in file:
                path2jsons.append(os.path.join(path, file))    
                
    return path2jsons

In [20]:
path2jsons_dev = get_jsons(path='/aimlx/Datasets/TEDLIUM_release1/dev/json')

In [99]:
extract_kw(keyword='people', path2jsons=path2jsons, path2kw_db='/aimlx/Datasets/TEDLIUM_release1/kw_db')

In [33]:
from collections import Counter

def word_count(path2json, output):
    word_count = Counter({})
    for path in tqdm(path2json):
        with open(path) as json_file:
            data = json.load(json_file)
            word_dict = dict(pd.Series(data['transcript'].split()).value_counts())
            word_dict = {k: float(v) for k, v in word_dict.items()}
            word_count = word_count + Counter(word_dict)
    
    word_count = dict(word_count)
    with open(output, 'w') as fp:
        json.dump(word_count, fp)

In [34]:
word_count(path2jsons_dev, '/aimlx/Datasets/TEDLIUM_release1/dev/dev_word_count.json')

100%|██████████| 8/8 [00:00<00:00, 24.31it/s]


In [35]:
_ = word_count('/aimlx/Datasets/TEDLIUM_release1/dev/json/BlaiseAguerayArcas_2007.json')

TypeError: word_count() missing 1 required positional argument: 'output'

In [36]:
_ = dict(df.value_counts())

In [37]:
os.getcwd()

'/aimlx/kws/kws_db_creation'

In [45]:
with open(os.path.join(os.getcwd(), 'overall_word_count.json' )) as json_file:
    data = json.load(json_file)

In [46]:
data = sorted(data.items(), key=operator.itemgetter(1), reverse=True)

In [47]:
data

[('the', 68116.0),
 ('and', 47518.0),
 ('to', 38736.0),
 ('of', 37390.0),
 ('a', 34068.0),
 ('that', 27099.0),
 ('in', 24561.0),
 ('is', 22081.0),
 ('i', 21518.0),
 ('you', 20832.0),
 ('we', 17911.0),
 ('it', 17294.0),
 ('this', 16711.0),
 ('so', 11769.0),
 ('was', 9202.0),
 ('for', 8913.0),
 ('have', 8821.0),
 ('are', 8536.0),
 ('they', 8495.0),
 ('on', 8452.0),
 ('but', 7827.0),
 ('what', 7808.0),
 ('with', 7594.0),
 ('about', 7109.0),
 ("it's", 7006.0),
 ('can', 6816.0),
 ('one', 6814.0),
 ('be', 6375.0),
 ('as', 6195.0),
 ('all', 6156.0),
 ('not', 5912.0),
 ('at', 5901.0),
 ('do', 5895.0),
 ('there', 5181.0),
 ('people', 5144.0),
 ('if', 5072.0),
 ('like', 4994.0),
 ('from', 4827.0),
 ('my', 4799.0),
 ("'s", 4770.0),
 ('these', 4593.0),
 ('know', 4474.0),
 ('now', 4461.0),
 ('just', 4382.0),
 ('very', 4314.0),
 ('an', 4247.0),
 ('he', 4187.0),
 ('or', 4107.0),
 ('because', 4088.0),
 ('out', 4048.0),
 ('when', 3926.0),
 ('our', 3841.0),
 ('up', 3827.0),
 ('which', 3610.0),
 ('think'